In [103]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.linear_model import LinearRegression
from statsmodels.iolib.summary2 import summary_col

### Data Upload

In [106]:
data = pd.read_csv("/Users/huangduo/Desktop/Machine Learning/HW1/morg-2014-emp.csv")

/var/folders/69/fws7qnp96rx9nd3jbrt3d3jw0000gn/T/ipykernel_9296/3512069760.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("/Users/huangduo/Desktop/Machine Learning/HW1/morg-2014-emp.csv")


In [108]:
business_ops_codes = [500, 510, 520, 530, 540, 565, 600, 630,
                      640, 650, 700, 710, 725, 726, 735, 740]

In [110]:
data_occ = data[data['occ2012'].isin(business_ops_codes)]
data_occ

,Unnamed: 0,hhid,intmonth,stfips,weight,earnwke,uhours,grade92,race,ethnic,...,ownchild,chldpres,prcitshp,state,ind02,occ2012,class,unionmme,unioncov,lfsr94
0,3,2600310997690,January,AL,3151.6801,1692.00,40,43,1,NaN,...,0,0,"Native, Born In US",63,Employment services (5613),630,"Private, For Profit",No,No,Employed-At Work
184,424,69997206074239,January,AK,374.8012,1057.00,42,39,1,NaN,...,1,3,"Native, Born In US",94,Support activities for mining (213),530,"Private, For Profit",No,No,Employed-At Work
241,534,700209900646154,January,AK,332.7025,1557.69,40,41,1,NaN,...,0,0,"Native, Born In US",94,"Justice, public order, and safety activities (...",740,Government - Federal,No,No,Employed-At Work
300,668,340446097003209,January,AZ,4272.6777,1634.61,40,42,1,NaN,...,1,1,"Native, Born In US",86,** Construction (23),710,"Private, For Profit",No,No,Employed-At Work
344,759,70447007503540,January,AZ,4431.6759,817.00,40,39,1,NaN,...,0,0,"Native, Born In US",86,Insurance carriers and related activities (524),540,"Private, For Profit",No,No,Employed-At Work
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149001,316451,185610760680506,December,WI,2072.8918,1500.00,40,39,1,NaN,...,0,0,"Native, Born In US",3,Administration of economic programs and space ...,530,Government - Federal,No,No,Employed-At Work
149070,316578,155654140100907,December,WI,3814.6828,1211.53,45,42,1,NaN,...,0,0,"Foreign Born, US Cit By Naturalization",3,Insurance carriers and related activities (524),540,"Private, For Profit",No,No,Employed-At Work
149083,316602,500010413026567,December,WI,4371.4088,1134.61,44,43,4,NaN,...,0,0,"Native, Born In US",3,"**** Data processing, hosting, and related ser...",740,"Private, For Profit",No,No,Employed-At Work
149122,316664,103165050066104,December,WI,3948.9255,1700.00,50,40,1,NaN,...,0,0,"Native, Born In US",3,"Structural metals, and tank and shipping conta...",530,"Private, For Profit",No,No,Employed-At Work


### Construct Variables

In [113]:
data_occ = data_occ.copy()

In [115]:
data_occ.loc[:, 'hourly_earn'] = data_occ['earnwke'] / data_occ['uhours']
data_occ.loc[:, 'education_years'] = data_occ['grade92']
data_occ.loc[:, 'experience'] = data_occ['age'] - data_occ['education_years'] - 6
data_occ.loc[data_occ['experience'] <= 0, 'experience'] = 0
data_occ.loc[:, 'female'] = (data_occ['sex'] == 2).astype(int)
data_occ.loc[:, 'ethnic_1'] = (data_occ['ethnic'] == 1).astype(int)

data_occ.loc[:, 'union'] = data_occ['unionmme'].map(lambda x: x.strip() if isinstance(x, str) else x)
data_occ.loc[:, 'union'] = data_occ['union'].map({"Yes": 1, "No": 0})

data_occ.loc[:, 'unioncov'] = data_occ['unioncov'].map(lambda x: x.strip() if isinstance(x, str) else x)
data_occ.loc[:, 'unioncov'] = data_occ['unioncov'].map({"Yes": 1, "No": 0})

data_occ.loc[:, 'lfsr94'] = data_occ['lfsr94'].map(lambda x: x.strip() if isinstance(x, str) else x)
data_occ.loc[:, 'lfsr94'] = data_occ['lfsr94'].map({"Employed-At Work": 1, "Employed-Absent": 2}).astype("Int64")

In [117]:
recode_prcitshp = {
    "Native, Born In US": 1,
    "Native, Born in PR or US Outlying Area": 2,
    "Native, Born Abroad Of US Parent(s)": 3,
    "Foreign Born, US Cit By Naturalization": 4,
    "Foreign Born, Not a US Citizen": 5}
data_occ['prcitshp_recode'] = data_occ['prcitshp'].map(lambda x: x.strip() if isinstance(x, str) else x)
data_occ['prcitshp_recode'] = data_occ['prcitshp_recode'].map(recode_prcitshp)

recode_class = {
    "Private, Nonprofit": 1,
    "Private, For Profit": 2,
    "Government - State": 3,
    "Foreign Born, Government - Local": 4,
    "Government - Federal": 5}
data_occ['class_recode'] = data_occ['class'].map(lambda x: x.strip() if isinstance(x, str) else x)
data_occ['class_recode'] = data_occ['class_recode'].map(recode_class)

In [119]:
data_occ

,Unnamed: 0,hhid,intmonth,stfips,weight,earnwke,uhours,grade92,race,ethnic,...,unioncov,lfsr94,hourly_earn,education_years,experience,female,ethnic_1,union,prcitshp_recode,class_recode
0,3,2600310997690,January,AL,3151.6801,1692.00,40,43,1,NaN,...,0.0,1,42.300000,43,0,1,0,0,1,2.0
184,424,69997206074239,January,AK,374.8012,1057.00,42,39,1,NaN,...,0.0,1,25.166667,39,3,0,0,0,1,2.0
241,534,700209900646154,January,AK,332.7025,1557.69,40,41,1,NaN,...,0.0,1,38.942250,41,10,1,0,0,1,5.0
300,668,340446097003209,January,AZ,4272.6777,1634.61,40,42,1,NaN,...,0.0,1,40.865250,42,0,1,0,0,1,2.0
344,759,70447007503540,January,AZ,4431.6759,817.00,40,39,1,NaN,...,0.0,1,20.425000,39,0,1,0,0,1,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149001,316451,185610760680506,December,WI,2072.8918,1500.00,40,39,1,NaN,...,0.0,1,37.500000,39,0,1,0,0,1,5.0
149070,316578,155654140100907,December,WI,3814.6828,1211.53,45,42,1,NaN,...,0.0,1,26.922889,42,7,1,0,0,4,2.0
149083,316602,500010413026567,December,WI,4371.4088,1134.61,44,43,4,NaN,...,0.0,1,25.786591,43,0,0,0,0,1,2.0
149122,316664,103165050066104,December,WI,3948.9255,1700.00,50,40,1,NaN,...,0.0,1,34.000000,40,4,0,0,0,1,2.0


### Establish OLS Regression

In [121]:
y = data_occ['hourly_earn']

In [123]:
x1 = sm.add_constant(data_occ[['education_years']])
model1 = sm.OLS(y, x1).fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:            hourly_earn   R-squared:                       0.091
Model:                            OLS   Adj. R-squared:                  0.091
Method:                 Least Squares   F-statistic:                     392.8
Date:                Sun, 02 Mar 2025   Prob (F-statistic):           2.16e-83
Time:                        20:17:09   Log-Likelihood:                -16389.
No. Observations:                3922   AIC:                         3.278e+04
Df Residuals:                    3920   BIC:                         3.279e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const             -84.4484      5.722    -

In [125]:
x2 = sm.add_constant(data_occ[['education_years', 'age', 'experience', 'female']])
model2 = sm.OLS(y, x2).fit()
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:            hourly_earn   R-squared:                       0.185
Model:                            OLS   Adj. R-squared:                  0.184
Method:                 Least Squares   F-statistic:                     222.1
Date:                Sun, 02 Mar 2025   Prob (F-statistic):          4.68e-172
Time:                        20:17:11   Log-Likelihood:                -16175.
No. Observations:                3922   AIC:                         3.236e+04
Df Residuals:                    3917   BIC:                         3.239e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const             -97.5536      5.679    -

In [127]:
x3 = sm.add_constant(data_occ[['education_years', 'age', 'experience',
                               'female', 'union', 'ethnic_1']])
x3.replace([np.inf, -np.inf], np.nan, inplace=True)
x3 = x3.infer_objects(copy=False)
y_x3 = y.loc[x3.index]
model3 = sm.OLS(y_x3, x3).fit()
print(model3.summary())

                            OLS Regression Results                            
Dep. Variable:            hourly_earn   R-squared:                       0.186
Model:                            OLS   Adj. R-squared:                  0.184
Method:                 Least Squares   F-statistic:                     148.7
Date:                Sun, 02 Mar 2025   Prob (F-statistic):          2.10e-170
Time:                        20:17:12   Log-Likelihood:                -16173.
No. Observations:                3922   AIC:                         3.236e+04
Df Residuals:                    3915   BIC:                         3.240e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const             -96.0120      5.746    -

/var/folders/69/fws7qnp96rx9nd3jbrt3d3jw0000gn/T/ipykernel_9296/3140538290.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x3.replace([np.inf, -np.inf], np.nan, inplace=True)


In [143]:
data_occ.loc[:, 'experience_sq'] = data_occ['experience']**2
data_occ.loc[:, 'female_ethnic'] = data_occ['female'] * data_occ['ethnic']
data_occ.loc[:, 'education_experience'] = data_occ['education_years'] * data_occ['experience']
x4 = sm.add_constant(data_occ[['education_years', 'age', 'experience', 'experience_sq',
                                 'female', 'union', 'ethnic_1', 'education_experience']])
x4.replace([np.inf, -np.inf], np.nan, inplace=True)
x4 = x4.dropna()
y_x4 = y.loc[x4.index]
model4 = sm.OLS(y_x4, x4).fit()
print(model4.summary())

                            OLS Regression Results                            
Dep. Variable:            hourly_earn   R-squared:                       0.186
Model:                            OLS   Adj. R-squared:                  0.185
Method:                 Least Squares   F-statistic:                     112.0
Date:                Sun, 02 Mar 2025   Prob (F-statistic):          5.33e-169
Time:                        20:30:28   Log-Likelihood:                -16172.
No. Observations:                3922   AIC:                         3.236e+04
Df Residuals:                    3913   BIC:                         3.242e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                 -102.2453 

/var/folders/69/fws7qnp96rx9nd3jbrt3d3jw0000gn/T/ipykernel_9296/2302588406.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x4.replace([np.inf, -np.inf], np.nan, inplace=True)


In [147]:
Regression_table = summary_col(
    [model1, model2, model3, model4],
    model_names=["(1)", "(2)", "(3)", "(4)"],
    stars=True,
    float_format="%0.3f",
    regressor_order=['education_years', 'age', 'experience', 'experience_sq',
                     'female', 'union', 'ethnic_1', 'education_experience'],
    drop_omitted=True,
    info_dict={'N': lambda x: f"{int(x.nobs)}"})

print(Regression_table)


                       (1)       (2)       (3)       (4)   
-----------------------------------------------------------
education_years      2.694*** 2.587***  2.553***  2.691*** 
                     (0.136)  (0.136)   (0.137)   (0.161)  
age                           0.521***  0.521***  0.534*** 
                              (0.032)   (0.032)   (0.035)  
experience                    -0.566*** -0.571*** 1.364    
                              (0.086)   (0.086)   (1.328)  
experience_sq                                     0.004    
                                                  (0.015)  
female                        -4.399*** -4.407*** -4.418***
                              (0.486)   (0.486)   (0.485)  
union                                   -0.526    -0.528   
                                        (1.047)   (1.047)  
ethnic_1                                -2.220*   -2.088   
                                        (1.290)   (1.292)  
education_experience                   

### Defining the Evaluation Function

In [149]:
def get_rmse(actual, pred):
    return np.sqrt(mean_squared_error(actual, pred))

def get_bic(model_result):
    return model_result.bic

def cross_val_rmse(X, y, k=5):
    lr = LinearRegression()
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    mse_scores = cross_val_score(lr, X, y, scoring='neg_mean_squared_error', cv=kf)
    rmse_scores = np.sqrt(-mse_scores)
    return rmse_scores.mean()

### Model Evaluation

In [152]:
pred1 = model1.predict(x1)
pred2 = model2.predict(x2)
pred3 = model3.predict(x3)
pred4 = model4.predict(x4)

rmse1 = get_rmse(y, pred1)
rmse2 = get_rmse(y, pred2)
rmse3 = get_rmse(y_x3, pred3)
rmse4 = get_rmse(y_x4, pred4)

bic1 = get_bic(model1)
bic2 = get_bic(model2)
bic3 = get_bic(model3)
bic4 = get_bic(model4)

cv_rmse1 = cross_val_rmse(x1.drop(columns='const', errors='ignore'), y.loc[x1.index], k=5)
cv_rmse2 = cross_val_rmse(x2.drop(columns='const', errors='ignore'), y.loc[x2.index], k=5)
cv_rmse3 = cross_val_rmse(x3.drop(columns='const', errors='ignore'), y_x3, k=5)
cv_rmse4 = cross_val_rmse(x4.drop(columns='const', errors='ignore'), y_x4, k=5)

print("Model 1 - RMSE:", rmse1, "BIC:", bic1, "CV RMSE:", cv_rmse1)
print("Model 2 - RMSE:", rmse2, "BIC:", bic2, "CV RMSE:", cv_rmse2)
print("Model 3 - RMSE:", rmse3, "BIC:", bic3, "CV RMSE:", cv_rmse3)
print("Model 4 - RMSE:", rmse4, "BIC:", bic4, "CV RMSE:", cv_rmse4)

Model 1 - RMSE: 15.795509858253515 BIC: 32793.99106483465 CV RMSE: 15.60939514482277
Model 2 - RMSE: 14.958124537233896 BIC: 32391.54271819907 CV RMSE: 14.754603384328405
Model 3 - RMSE: 14.951989474784785 BIC: 32404.87356212982 CV RMSE: 14.755789964898716
Model 4 - RMSE: 14.9451753335024 BIC: 32417.846677889294 CV RMSE: 14.772907327045981


In [154]:
results = pd.DataFrame({
    'Model': ['Model 1', 'Model 2', 'Model 3', 'Model 4'],
    'RMSE': [rmse1, rmse2, rmse3, rmse4],
    'BIC': [bic1, bic2, bic3, bic4],
    'CV_RMSE': [cv_rmse1, cv_rmse2, cv_rmse3, cv_rmse4]})

print(results)

     Model       RMSE           BIC    CV_RMSE
0  Model 1  15.795510  32793.991065  15.609395
1  Model 2  14.958125  32391.542718  14.754603
2  Model 3  14.951989  32404.873562  14.755790
3  Model 4  14.945175  32417.846678  14.772907


### Descriptive Statistics

In [52]:
num_cols = ['hourly_earn', 'education_years', 'age', 'experience',
            'female', 'union', 'ethnic_1',]
print(data_occ[num_cols].describe())

       hourly_earn  education_years          age   experience       female  \
count  3922.000000      3922.000000  3922.000000  3922.000000  3922.000000   
mean     28.842154        42.057879    41.522183     2.586435     0.583376   
std      16.570141         1.856484    11.951643     4.586976     0.493062   
min       0.000500        32.000000    16.000000     0.000000     0.000000   
25%      17.933333        40.000000    31.000000     0.000000     0.000000   
50%      25.277797        43.000000    41.000000     0.000000     1.000000   
75%      36.057500        43.000000    51.000000     4.000000     1.000000   
max     461.538000        46.000000    64.000000    19.000000     1.000000   

          ethnic_1  
count  3922.000000  
mean      0.036461  
std       0.187458  
min       0.000000  
25%       0.000000  
50%       0.000000  
75%       0.000000  
max       1.000000  


In [139]:
data_occ.loc[data_occ['hourly_earn'] >= 1, ['hourly_earn', 'education_years', 'age', 'experience',
                                            'female', 'union', 'ethnic_1']].describe()

,hourly_earn,education_years,age,experience,female,ethnic_1
count,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000
mean,28.871579,42.056662,41.513527,2.587289,0.582950,0.036498
std,16.552969,1.856852,11.954223,4.588217,0.493134,0.187550
min,1.346000,32.000000,16.000000,0.000000,0.000000,0.000000
25%,17.961500,40.000000,31.000000,0.000000,0.000000,0.000000
50%,25.331068,43.000000,41.000000,0.000000,1.000000,0.000000
75%,36.057500,43.000000,51.000000,4.000000,1.000000,0.000000
max,461.538000,46.000000,64.000000,19.000000,1.000000,1.000000
